Importación de librerías

In [0]:
import pandas as pd

In [66]:
!pip3 install h2o

Inicio H2O

In [67]:
import h2o
h2o.init(nthreads = -1, max_mem_size = 8)

from h2o.automl import H2OAutoML, get_leaderboard

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 hours 24 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,1 month and 4 days
H2O cluster name:,H2O_from_python_unknownUser_ztfple
H2O cluster total nodes:,1
H2O cluster free memory:,7.993 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


Importación y pequeña preparación de Pandas

In [0]:
df = pd.read_csv("/content/drive/My Drive/Csv4.csv").reset_index(drop=True)

In [0]:
del df["Unnamed: 0"]

In [70]:
df.head()

,Host Country,Year,Region,Athlete,Sport,Medal_Gold,Medal_Silver,Medal_Bronze,Medals
0,ESP,1992,Kuwait,32,7,0,0,0,0
1,ESP,1992,Niger,3,1,0,0,0,0
2,ESP,1992,Nigeria,55,8,0,7,4,11
3,ESP,1992,North Korea,64,12,4,0,6,10
4,ESP,1992,Norway,83,17,2,20,1,23


In [71]:
df.shape

(1365, 9)

Preparación de la maquinaria de Machine Learning

In [49]:
data = h2o.H2OFrame(df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
splits = data.split_frame(ratios=[0.7, 0.15], seed=1)  

In [0]:
train = splits[0]
valid = splits[1]
test = splits[2]

In [0]:
y = 'Medals'
x = list(data.columns)

x.remove("Medals")
x.remove("Medal_Gold")
x.remove("Medal_Silver")
x.remove("Medal_Bronze")

In [0]:
x

['Host Country', 'Year', 'Region', 'Athlete', 'Sport']

In [0]:
aml = H2OAutoML(max_models = 20, nfolds=10)
aml.train(x = x, y = y, training_frame = train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


Métricas

In [0]:
aml.leaderboard.head(1)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_AutoML_20200310_010756,61.722,7.85633,61.722,3.27049,nan


In [0]:
lb = get_leaderboard(aml, extra_columns='ALL')
lb.head(rows=lb.nrows)

Guardado del modelo

In [0]:
h2o.save_model(aml.leader, path = "model")

'/content/model/StackedEnsemble_BestOfFamily_AutoML_20200310_010756'

Predicciones

In [0]:
aml = h2o.load_model("/content/drive/My Drive/StackedEnsemble_BestOfFamily_AutoML_20200310_010756")

In [73]:
pred = aml.predict(test)
pred[pred < 0] = 0
pred = pred.round()
pred.head(3)

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict
0
0
179


In [0]:
predDef = pred.as_data_frame()


Predicciones de los Juegos Olímpicos de Tokio

In [75]:
df.head()

,Host Country,Year,Region,Athlete,Sport,Medal_Gold,Medal_Silver,Medal_Bronze,Medals
0,ESP,1992,Kuwait,32,7,0,0,0,0
1,ESP,1992,Niger,3,1,0,0,0,0
2,ESP,1992,Nigeria,55,8,0,7,4,11
3,ESP,1992,North Korea,64,12,4,0,6,10
4,ESP,1992,Norway,83,17,2,20,1,23


In [0]:
df.drop(columns=["Medals","Medal_Gold","Medal_Silver","Medal_Bronze"], inplace=True)
df2 = df.loc[df["Year"]==2016].sort_values(by=["Year"]).reset_index(drop=True)
df2["Year"] = df2["Year"].replace({2016: 2020})

In [77]:
df2.head()

,Host Country,Year,Region,Athlete,Sport
0,BRA,2020,Oman,4,2
1,BRA,2020,Albania,6,3
2,BRA,2020,Algeria,64,13
3,BRA,2020,Andorra,4,4
4,BRA,2020,Angola,26,7


In [78]:
data2 = h2o.H2OFrame(df2)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [79]:
pred = aml.predict(data2)
pred[pred < 0] = 0
pred = pred.round()

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [0]:
pred = pred.as_data_frame()


In [0]:
dfMain = df2.join(pred)

In [82]:
dfMain

,Host Country,Year,Region,Athlete,Sport,predict
0,BRA,2020,Oman,4,2,1
1,BRA,2020,Albania,6,3,1
2,BRA,2020,Algeria,64,13,3
3,BRA,2020,Andorra,4,4,0
4,BRA,2020,Angola,26,7,0
...,...,...,...,...,...,...
198,BRA,2020,Somalia,2,1,1
199,BRA,2020,Turkey,100,21,11
200,BRA,2020,Turkmenistan,9,5,0
201,BRA,2020,Swaziland,2,1,0


In [83]:
dfMain.loc[asd["Region"]=="Spain"]

,Host Country,Year,Region,Athlete,Sport,predict
194,BRA,2020,Spain,307,28,56


In [84]:
dfMain.loc[asd["Region"]=="USA"]

,Host Country,Year,Region,Athlete,Sport,predict
177,BRA,2020,USA,555,33,258


Exportación de resultados

In [0]:
from google.colab import files

dfMain.to_csv('df.csv')
files.download('df.csv')